In [103]:
import pandas as pd
import numpy as np
# from src.encoder import encode, decode
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from src.ANN import *
from sklearn.model_selection import KFold
from src.metrics import confusion_matrix

In [104]:
df = pd.read_csv('Iris.csv')
df = df.drop(['Id'],axis=1)

x_train = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
y_train = df['Species'].map({
    "Iris-setosa": 0,
    "Iris-versicolor": 1,
    "Iris-virginica": 2
}).astype(int)

In [105]:
labelled_y = []
for i in y_train:
    a = [0,0,0]
    a[i] = 1
    labelled_y.append(a)  

In [106]:
X = np.array(x_train)
Y = np.array(labelled_y)

In [107]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.20, random_state=420)

In [108]:
model = Sequential(random_state=42069)
model.add(Dense(units=16, input_dim=4, activation_function="linear"))
model.add(Dense(units=8, activation_function="relu"))
model.add(Dense(units=4, activation_function="linear"))
model.add(Dense(units=3, activation_function="relu"))
model.add(Dense(units=3, activation_function="sigmoid"))
model.compile("sum_squared_error", 0.0001, 0.001)


In [109]:
model.fit(X, Y, batch_size=1, epoch=50)
model.summary()

MODEL INFO
units : 16  ||  activation function : linear  ||  input_dim : 4
weights :
[0.0021933410382560885, -0.001022520693670169, 0.1306102670359696, 0.45901957098035373, -0.05332000081754657]
[0.003216871513581527, -0.08049051877345345, -0.07279540987677409, 0.45282037841910594, -0.08227728326018513]
[0.0037187627308950246, 0.11005287775878177, -0.3820373436917025, 0.3470687824994323, -0.03452313106130221]
[0.0007357845872829988, 0.17791445671518985, 0.17574692788445004, 0.001839631673286147, 0.09413683878293158]
[-0.0019884454304422805, 0.2587917679081659, -0.046048028960387205, -0.14827899838925532, -0.20562779309869367]
[0.004351738247555762, -0.1563850042388732, 0.18233917173274297, -0.07081485512170174, -0.1969065348765972]
[0.002901088796596139, 0.17377428135343054, 0.22347174939102749, -0.03804289124847361, -0.47614184670281795]
[0.006714461037831053, -0.02467156694110924, -0.004511892245086892, -0.30018759843047876, 0.3584046854278444]
[0.0021195658795323358, -0.448486172294

### Cell dibawah hanya untuk liat akurasi model saja

In [110]:
model.fit(train_x, train_y, batch_size=1, epoch=500)

In [111]:
y_pred = model.predict(test_x)

In [112]:
decode_test_y = []
for i in test_y:
  decode_test_y.append(np.argmax(i))
print(decode_test_y)


[1, 2, 2, 0, 0, 1, 0, 1, 2, 1, 2, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 1, 0, 1, 1, 0, 0, 1, 1]


In [113]:
decode_pred_y = []
for i in y_pred:
  decode_pred_y.append(np.argmax(i))
print(decode_pred_y)

[2, 2, 2, 0, 0, 1, 0, 1, 2, 1, 2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 1, 0, 1, 1, 0, 0, 1, 1]


In [114]:
accuracy_score(y_pred=decode_pred_y, y_true=decode_test_y)

0.9333333333333333

In [115]:
print("confusion matrix:")
print(confusion_matrix(decode_pred_y, decode_test_y))

confusion matrix:
{0: 0, 1: 1, 2: 2}
[[13  0  0]
 [ 0  9  0]
 [ 0  2  6]]


### Cell dibawah untuk cross validation

In [116]:
accurations = []
num_data = len(X)
k_fold = 10
# kf = KFold(n_splits=k_fold)
# for train, test in kf.split(X):
#   train_x, test_x = X[train], X[test]
#   train_y, test_y = Y[train], Y[test]
#   print(train_x.shape)
#   print(test_x.shape)
test_size = num_data//k_fold
test_slice_x = []
test_slice_y = []
remainder_x = []
remainder_y = []

# X[:test_size]
# test_size
# num_data

for i in range(k_fold):
  if i == 0:
    test_slice_x, remainder_x = X[:test_size], X[test_size:]
    test_slice_y, remainder_y = Y[:test_size], Y[test_size:]
  else:
    start = (i-1)*(test_size)
    end = min(start + (test_size), num_data)
    remainder_x[start:end], test_slice_x = test_slice_x, remainder_x[start:end].copy()
    remainder_y[start:end], test_slice_y = test_slice_y, remainder_y[start:end].copy()
  # print(test_slice_y)
  # # print(y_test)
  # print('test_x: ', test_slice_x, 'train_x:', remainder_x)
  # print('test_y: ', test_slice_y, 'train_y: ', remainder_y)
  model.fit(remainder_x, remainder_y, batch_size=1, epoch=5)
  y_pred = model.predict(test_slice_x)
  decode_test_y = []
  for i in test_slice_y:
    decode_test_y.append(np.argmax(i))
  decode_pred_y = []
  for i in y_pred:
    decode_pred_y.append(np.argmax(i))
  print(decode_pred_y)
  accurations.append(accuracy_score(y_pred=decode_pred_y, y_true=decode_test_y))
  model.reset()

print(decode_pred_y)
print(decode_test_y)
  
print("confusion matrix:")
print(confusion_matrix(decode_pred_y, decode_test_y))


[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
confusion matrix:
{2: 0}
[[15]]


In [117]:
accurations

[1.0,
 1.0,
 1.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.3333333333333333,
 0.9333333333333333,
 0.6666666666666666,
 1.0]

In [118]:
np.average(accurations)

0.6266666666666667